### GDP Components

In [19]:
import requests
import pandas as pd
import config   ## File with API key
api_key = config.bea_key

# Components of request
base = 'https://www.bea.gov/api/data/?&UserID={}'.format(api_key)
m = '&method=GetData'
dsn = '&datasetname=NIPA'
ind = '&TableId=2'
freq = '&Frequency=Q'
year = '&Year=LAST12'
fmt = '&ResultFormat=json'

# Combined url for request
url = '{}{}{}{}{}{}{}'.format(base, m, dsn, ind, freq, year, fmt)
r = requests.get(url).json()
df = pd.DataFrame(r['BEAAPI']['Results']['Data'])
df['date'] = pd.to_datetime(df['TimePeriod'])
codes = ['A191RL', 'DPCERY', 'A006RY', 'A019RY', 'A822RY']
ind = ['date', 'SeriesCode']
df = df[df['SeriesCode'].isin(codes)].set_index(ind)['DataValue'].unstack(1)

# Record strings for later use in chart label
date = '{} Q{}'.format(df.index[-1].year, df.index[-1].quarter)
ch = df['A191RL'][-1]  # GDP in growth
label = '{}: Real GDP growth: {}\%'.format(date, ch)

### GDP contributions by industry

In [39]:
import requests
import pandas as pd
import config   ## File with API key
import roman
api_key = config.bea_key

# Components of request
base = 'https://www.bea.gov/api/data/?&UserID={}'.format(api_key)
m = '&method=GetData'
dsn = '&datasetname=GDPbyIndustry'
ind = '&TableId=13'
freq = '&Frequency=Q'
year = '&Year=2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016'
fmt = '&ResultFormat=json'
indus = '&Industry=ALL'

# Combined url for request
url = '{}{}{}{}{}{}{}{}'.format(base, m, dsn, year, indus, ind, freq, fmt)
r = requests.get(url).json()
df = pd.DataFrame(r['BEAAPI']['Results']['Data'])
df.loc[:,'TimePeriod'] = ['{}Q{}'.format(
    df.iloc[n]['Year'], roman.fromRoman(df.iloc[n]['Quarter'])) 
    for n in df.index.values]
df['date'] = pd.to_datetime(df['TimePeriod'])
codes = ['11', '21', '22', '23', '33DG', '33ND', '42', '44RT', 
         '48TW', '51', '52', '53', '54', '55', '56', '61', '62', 
         '71', '72', '81', 'GF', 'GSL']
ind = ['date', 'IndustrYDescription']
df2 = df[df['Industry'].isin(codes)].set_index(ind)['DataValue'].unstack(1)
df2.to_csv('Industry_contributions_to_GDP.csv')